In [ ]:
# Install required libraries
%pip install llama-index-llms-openai
!pip install llama-index

In [ ]:
# Import necessary libraries
import logging
import sys
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.retrievers import VectorIndexRetriever

# Configure logging
logging.basicConfig(stream=sys.stdout, level=logging.ERROR)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
from llama_index.core import Settings

# Configure Ollama LLM
ollama_llm = Ollama(
    model="llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

Settings.llm = ollama_llm
Settings.embed_model = ollama_embedding

In [ ]:
# Load documents into the index

documents = SimpleDirectoryReader(input_files=['../data/paul_graham_essay3.txt']).load_data()
# documents = SimpleDirectoryReader(input_files=['../data/2022 Q3 AAPL.pdf']).load_data()
index = VectorStoreIndex.from_documents(documents)


In [ ]:
from llama_index.core.tools import QueryEngineTool


# Define a summarization query engine
list_tool = QueryEngineTool.from_defaults(
    query_engine=index.as_query_engine(),
    description="Useful for summarization questions related to the data source"
)

# Define a semantic search query engine
vector_tool = QueryEngineTool.from_defaults(
    query_engine=index.as_query_engine(),
    description="Useful for retrieving specific context related to the data source"
)


In [ ]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import PydanticSingleSelector

from llama_index.core.selectors import LLMSingleSelector

# Create an LLM-based single selector
selector = LLMSingleSelector.from_defaults(llm=ollama_llm)

# Create a router query engine
router_query_engine = RouterQueryEngine(
    selector=selector,
    query_engine_tools=[list_tool, vector_tool],
)



In [ ]:
# Query the router for summarization
response = router_query_engine.query("Summarize the key points from the essay.")
print("Summarization Response:")
print(response)

# Query the router for semantic search
response = router_query_engine.query("What did the author do at Viaweb and Interleaf?")
print("Semantic Search Response:")
print(response)
